# 레스토랑 데이터를 정제해서 mysql에 집어 넣는다.
## 절대 실행하지마세요 이미 완료된 작업

In [7]:
import graphlab as gl
import pymysql as sql
import pymysql.cursors


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# if connection:
#     connection.close()

kakao_host ='10.198.23.243' # 카카오 내 호스트
kakao_user = 'user'
google_host = '35.229.56.244' # 구글 클라우드 호스트
google_user = 'root'
connection = sql.connect(host=kakao_host,
                         user=kakao_user,
                         password='8888',
                         db='ITM',
                         charset='utf8mb4',
                         cursorclass=pymysql.cursors.DictCursor)
connection

In [8]:
rest = gl.SFrame('./dataset/restaurant-dataset/')
rest.head(5)

Columns:
	address	str
	business_id	str
	categories	str
	city	str
	hours	dict
	is_open	int
	latitude	float
	longitude	float
	name	str
	postal_code	str
	review_count	int
	stars	float
	state	str
	Sunday	str
	Monday	str
	Tuesday	str
	Thursday	str
	Friday	str
	Saturday	str
	RestaurantsDelivery	int
	BusinessAcceptsCreditCards	int
	RestaurantsTakeOut	int
	Alcohol	int
	price	str

Rows: 5

Data:
+-------------------------------+------------------------+
|            address            |      business_id       |
+-------------------------------+------------------------+
|        211 W Monroe St        | bFzdJJ3wp3PZssNEsyU23g |
| 20235 N Cave Creek Rd, Ste... | 45bWSZtniwPRiqlivpS8Og |
|  3417 Derry Road E, Unit 103  | 6OuOZAok8ikONMS_T3EzXg |
|       1440 N. Dysart Ave      | 8-NRKkPY1UiFXW20WXKiXg |
|        209 Oakland Ave        | UTm5QZThPQlT35mkAcGOjg |
+-------------------------------+------------------------+
+-------------------------------+-------------+-------------------------------+
|           categories          |     city    |             hours             |
+-------------------------------+-------------+-------------------------------+
| Insurance, Financial Services |   Phoenix   |              None             |
|       Coffee & Tea, Food      |   Phoenix   | {'Monday': '5:30-20:0', 'T... |
|       Restaurants, Thai       | Mississauga |              None             |
|      Mexican, Restaurants     |   Avondale  | {'Monday': '0:0-0:0', 'Tue... |
| Flowers & Gifts, Gift Shop... |  Pittsburgh | {'Monday': '9:0-18:0', 'Tu... |
+-------------------------------+-------------+-------------------------------+
+---------+---------------+----------------+----------------------------+
| is_open |    latitude   |   longitude    |            name            |
+---------+---------------+----------------+----------------------------+
|    1    |   33.4499993  |  -112.0769793  |      Geico Insurance       |
|    1    |   33.6713751  |  -112.0300171  | The Coffee Bean & Tea Leaf |
|    1    |   43.7129464  |  -79.6327631   |        Thai One On         |
|    1    | 33.4481059352 | -112.341302074 |  Filiberto's Mexican Food  |
|    1    |   40.4414214  |  -79.9564571   |  Maggie & Stella's Gifts   |
+---------+---------------+----------------+----------------------------+
+-------------+--------------+-------+-------+-----------+-----------+-----------+
| postal_code | review_count | stars | state |   Sunday  |   Monday  |  Tuesday  |
+-------------+--------------+-------+-------+-----------+-----------+-----------+
|    85003    |      8       |  1.5  |   AZ  |           |           |           |
|    85024    |      63      |  4.0  |   AZ  | 6:30-19:0 | 5:30-20:0 | 5:30-20:0 |
|   L4T 1A8   |      7       |  2.0  |   ON  |           |           |           |
|    85323    |      40      |  2.5  |   AZ  |  0:0-0:0  |  0:0-0:0  |  0:0-0:0  |
|    15213    |      3       |  3.5  |   PA  |           |  9:0-18:0 |  9:0-18:0 |
+-------------+--------------+-------+-------+-----------+-----------+-----------+
+-----------+-----------+-----------+---------------------+-----+
|  Thursday |   Friday  |  Saturday | RestaurantsDelivery | ... |
+-----------+-----------+-----------+---------------------+-----+
|           |           |           |          0          | ... |
| 5:30-20:0 | 5:30-21:0 | 5:30-21:0 |          0          | ... |
|           |           |           |          0          | ... |
|  0:0-0:0  |  0:0-0:0  |  0:0-0:0  |          0          | ... |
|  9:0-18:0 |  9:0-17:0 | 10:0-17:0 |          0          | ... |
+-----------+-----------+-----------+---------------------+-----+
[5 rows x 24 columns]

In [24]:
def insert_database(rest):
    with connection.cursor() as cursor:
        # 레스토랑 - 카테고리 삽입
        def restaurant_category(rest_id, category_id):
            sql = """
            INSERT INTO `restaurant_categories` (`restaurant_id`, `category_id`)
            VALUES (%s, %s)
            """
            cursor.execute(sql, (rest_id, category_id))
            connection.commit()
        
        # 카테고리 확인 후 작업
        def insert_category(name, id):
            if name is None or name.strip() == '':
                return;
            name = name.lower().strip()
            
            sql = """
            SELECT `id` from categories where LOWER(`category_name`) = %s
            """
            cursor.execute(sql, name)
            result = cursor.fetchone()
            
            if result:
                category_id = result['id']
                restaurant_category(id, category_id)
            else:
                sql = """ 
                INSERT INTO `categories` (`category_name`)
                VALUES (%s)
                """
                result = cursor.execute(sql, (name, ))
                category_id = cursor.lastrowid
                connection.commit()
                restaurant_category(id, category_id)
        
        sql = """
        INSERT INTO `restaurants` 
        (`address`, `business_id`, `city`, `longitude`, `latitude`, `stars`, `name`, `postal_code`, `review_count`, `state`, `delivery_yn`, 
        `card_yn`, `takeout_yn`, `alcohol_yn`, `price_category`, `created_at`, `updated_at`)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, current_timestamp(), current_timestamp())
        """
        cursor.execute(sql, (rest['address'], rest['business_id'], rest['city'], rest['longitude'], rest['latitude'], rest['stars'], rest['name'],
                            rest['postal_code'], rest['review_count'], rest['state'], rest['RestaurantsDelivery'], rest['BusinessAcceptsCreditCards'],
                            rest['RestaurantsTakeOut'], rest['Alcohol'], rest['price'],))
        
        
        rest_id = cursor.lastrowid
        connection.commit()
        
        categories = rest['categories']
        if categories is not None:
            for category in categories.split(','):
                cate = category.lower().strip()
                insert_category(cate, rest_id)
            

In [25]:
from tqdm import tqdm_notebook as tqdm
for i in tqdm(rest):
    insert_database(i)

In [13]:
str(rest[1]['stars'])

'4.0'